In [1]:
!pip install langchain openai faiss-cpu tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.3 MB/s eta 0:00:00


get the openApi ket from openai accounts

In [2]:
import os
os.environ["OPENAI_API_KEY"]="YOUR_KEY"

In [3]:
#to upload support files
from google.colab import files
uploaded=files.upload()

Saving Ecommerce_FAQ_Chatbot_dataset.json to Ecommerce_FAQ_Chatbot_dataset.json


In [4]:
import json

# Load your uploaded JSON file
with open("Ecommerce_FAQ_Chatbot_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Check the first entry to understand structure
print(data['questions'][1])

{'question': 'What payment methods do you accept?', 'answer': 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.'}


In [5]:
#convert json into langchain documents  because LangChain’s retriever pipeline doesn’t work directly on raw JSON.
from langchain.schema import Document
docs=[]
for item in data['questions']:
    q=item.get("question","")
    a=item.get("answer","")
    content=f"Question:{q}\nAnswer:{a}"
    docs.append(Document(page_content=content))

print(f"Created {len(docs)} documents")

Created 79 documents


spplit into chunks


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
chunks=splitter.split_documents(docs)
print(f"split into {len(chunks)} chunks")

split into 79 chunks


In [7]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.1 MB/s eta 0:00:00


Facebook AI Similarity Search (FAISS)-it’s a library made by Facebook AI Research for fast, efficient vector search.

In [8]:
#create embeddings and store in Faiss
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
embeddings=OpenAIEmbeddings(model="text-embedding-3-small")
vectorStore=FAISS.from_documents(chunks, embeddings)

/tmp/ipython-input-2849870605.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings(model="text-embedding-3-small")


create the rag QA chain

Right now you have:

Chunks of your documents

Embeddings stored in FAISS

A language model (OpenAI GPT, etc.)

But they’re separate pieces.
The RAG chain ties them together so that:

User asks a question

Retriever (FAISS) finds the most relevant chunks from your dataset

LLM takes:

The question

The retrieved chunks as context

And generates a final answer grounded in your data







In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
llm=ChatOpenAI(model_name="gpt-4o",temperature=0)
qa_chain=RetrievalQA.from_chain_type(
    llm,
    retriever=vectorStore.as_retriever(search_kwargs={"k":3}),
    chain_type="stuff"
)

/tmp/ipython-input-3565763511.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(model_name="gpt-4o",temperature=0)


Test

In [10]:
query="what is your refund policy"
answer=qa_chain.run(query)
print(answer)

/tmp/ipython-input-2461348352.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer=qa_chain.run(query)


Our refund policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.


User interface


In [11]:
!pip install gradio

In [15]:
import gradio as gr

def chatbot_ui(query, history):
    answer = qa_chain.run(query)
    history = history + [(query, answer)]
    return history, history

custom_css = """
#chatbot {
    background-color: #f0f9ff;
    border-radius: 12px;
    padding: 4px;
}

.gradio-container {
    background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);
    font-family: 'Segoe UI', sans-serif;
}

textarea {
    border: 2px solid #ff6f61 !important;
    border-radius: 8px !important;
}

button {
    background-color: #ff6f61 !important;
    color: white !important;
    border-radius: 8px !important;
}
"""

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown(
        "<h1 style='text-align:center;color:#ff6f61;'>🛒 Ecommerce Support Chatbot</h1>"
    )
    chatbot = gr.Chatbot(elem_id="chatbot")
    msg = gr.Textbox(label="Ask a question")
    clear = gr.Button("Clear Chat")

    state = gr.State([])

    msg.submit(chatbot_ui, [msg, state], [chatbot, state])
    clear.click(lambda: None, None, chatbot)

demo.launch(share=True)


/tmp/ipython-input-2144989600.py:36: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(elem_id="chatbot")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c124a9bf3cf318a74e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
